# Projeto 2 - Ciência dos Dados

Nome: Bruno Meneghesso da Silva

Nome: Diogo Nobre de Araujo Cintra

___
# Classificador automático de sentimento para o Twitter


### Importando as bibliotecas

In [42]:
%%capture
!pip install tweepy
%matplotlib inline
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
from numpy import arange
from numpy import percentile
import numpy as np
import os
import re 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

___
## Autenticando no  Twitter

* Conta: ***@Ciencia dos dados***

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Capturando os dados do twitter:

Salvando os dados em uma planilha Excel:

___
### Classificando as mensagens na coragem

#### 0 = irrelevante
#### 1 = relevante

In [3]:
TREINAMENTO = pd.read_excel('Correios.xlsx',sheet_name = 'Treinamento')
TESTE = pd.read_excel('Correios.xlsx',sheet_name = 'Teste')

In [4]:
TREINAMENTO_RELEVANTE = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==1]
TREINAMENTO_IRRELEVANTE = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==0]

def clean(text):
    punctuation = '[!\-.:?;#$%&*_1234567890"]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [5]:
texto_relevante = " ".join(TREINAMENTO_RELEVANTE['Treinamento'])
texto_relevante_1 = clean(texto_relevante)
texto_relevante_2 = texto_relevante_1.split()
texto_relevante_3 = []

texto_irrelevante = " ".join(TREINAMENTO_IRRELEVANTE['Treinamento'])
texto_irrelevante_1 = clean(texto_irrelevante)
texto_irrelevante_2 = texto_irrelevante_1.split()
texto_irrelevante_3 = []

treinamento = " ".join(TREINAMENTO['Treinamento'])
treinamento_1 = clean(treinamento)
treinamento_2 = treinamento_1.split()
treinamento_3 = []

teste = " ".join(TESTE['Teste'])
teste_1 = clean(teste)
teste_2 = teste_1.split()
teste_3 = []

for e in range(len(texto_relevante_2)-1):
    if texto_relevante_2[e] != 'rt' and texto_relevante_2[e][0] != '@':     
        texto_relevante_3.append(texto_relevante_2[e])    
        
for e in range(len(texto_irrelevante_2)-1):
    if texto_irrelevante_2[e] != 'rt' and texto_irrelevante_2[e][0] != '@':     
        texto_irrelevante_3.append(texto_irrelevante_2[e]) 
        
for e in range(len(treinamento_2)-1):
    if treinamento_2[e] != 'rt' and treinamento_2[e][0] != '@':     
        treinamento_3.append(treinamento_2[e])
        
for e in range(len(teste_2)-1):
    if teste_2[e] != 'rt' and teste_2[e][0] != '@':     
        teste_3.append(teste_2[e])        
        
texto_relevante_4 = pd.Series(texto_relevante_3)
texto_irrelevante_4 = pd.Series(texto_irrelevante_3)
treinamento_4 = pd.Series(treinamento_3)
teste_4 = pd.Series(teste_3)

#### Probabilidade do tweet ser relevante

In [6]:
prob_relevante = len(TREINAMENTO.loc[TREINAMENTO["Classificacao"]==1])/len(TREINAMENTO["Treinamento"])
prob_relevante

0.4013377926421405

#### Probabilidade do tweet ser relevante

In [7]:
prob_irrelevante = len(TREINAMENTO.loc[TREINAMENTO["Classificacao"]==0])/len(TREINAMENTO["Treinamento"])
prob_irrelevante

0.5986622073578596

___
### Montando o Classificador Naive-Bayes

In [8]:
total_amostral_teste = len(teste_4)
total_amostral_treinamento = len(treinamento_4)
total_relevante = texto_relevante_4.value_counts()
total_irrelevante = texto_irrelevante_4.value_counts()

#Probabilidade da palavra ser relevante
prob_relevante_palavra = total_relevante/total_amostral_treinamento
prob_relevante_palavra

#Probabilidade da palavra ser irrelevante
prob_irrelevante_palavra = total_irrelevante/total_amostral_treinamento
prob_irrelevante_palavra
total_amostral_treinamento

6337

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [9]:
prob_relevante_teste = len(TESTE.loc[TESTE["Classificacao"]==1])/len(TESTE["Teste"])
prob_irrelevante_teste = len(TESTE.loc[TESTE["Classificacao"]==0])/len(TESTE["Teste"])

smoth = 1e-10 #é uma estimativa e deve mudar
vocabulario = 356000

# defini probabilidade de ser relevante:
def relevante(tweet):
    
    tweet_relevante_2 = clean(tweet)
    tweet_relevante_3 = tweet_relevante_2.split()
    tweet_relevante_4 = []
    
    for e in range(len(tweet_relevante_3)-1):
        
        if tweet_relevante_3[e] != 'rt' and tweet_relevante_3[e][0] != '@':     
            tweet_relevante_4.append(tweet_relevante_3[e])
            
    prob = 1
    
    for p in tweet_relevante_4:
        
        if p in prob_relevante_palavra:
            prob = prob*(prob_relevante_palavra[p]*total_relevante.size+smoth)/(total_relevante.size+smoth*vocabulario)*1000
        else:
            prob = prob*(smoth)/(total_relevante.size+smoth*vocabulario)*1000
            
    prob = prob * prob_relevante
    prob = math.log10(prob)
    #print(prob)
    return prob

# defini probabilidade de ser irrelevante:
def irrelevante(tweet):
    
    tweet_irrelevante_2 = clean(tweet)
    tweet_irrelevante_3 = tweet_irrelevante_2.split()
    tweet_irrelevante_4 = []
    
    for e in range(len(tweet_irrelevante_3)-1):
        
        if tweet_irrelevante_3[e] != 'rt' and tweet_irrelevante_3[e][0] != '@':     
            tweet_irrelevante_4.append(tweet_irrelevante_3[e])
            
    prob = 1
    
    for p in tweet_irrelevante_4:
        
        if p in prob_irrelevante_palavra:
            prob = prob*(prob_irrelevante_palavra[p]*total_irrelevante.size+smoth)/(total_irrelevante.size+smoth*vocabulario)*1000
        else:
            prob = prob*(smoth)/(total_irrelevante.size+smoth*vocabulario)*1000
            
    prob = prob * prob_irrelevante
    prob = math.log10(prob)
    #print(prob)
    return prob

In [10]:
def compara (tweet):
    
    r = relevante(tweet)
    nor = irrelevante(tweet)
    
    return r>nor

rev = 0
norev = 0

for e in TESTE["Teste"]:
    
    if compara(e):
        rev +=1
    else:
        norev+=1

In [32]:
print("Probabilidade de ser relevante segundo o algoritmo: {}%".format((rev/TESTE["Teste"].size)*100))
print("Probabilidade de ser irrelevante segundo o algoritmo: {}%".format((norev/TESTE["Teste"].size)*100))
"""
if rev/(TESTE["Teste"].size) < 0.9*prob_relevante_teste:
    print ("poucos relevantes {} vezes o esperado".format(rev/TESTE["Teste"].size/prob_relevante_teste))
    
elif rev/(TESTE["Teste"].size) > 1.1*prob_relevante_teste:
    print ("muitos relevantes {} vezes o esperado" .format(rev/TESTE["Teste"].size/prob_relevante_teste))
    
else:
    print ("ok: {} veze o esperado".format (rev/TESTE["Teste"].size/prob_relevante_teste))  
"""   
print("Probabilidade de ser relevante pela nossa classificação: {:.2f}%".format(prob_relevante_teste*100))
print("Probabilidade de ser irrelevante pela nossa classificação: {:.2f}%".format(prob_irrelevante_teste*100))
print("Acurácia do algoritmo: {:.2f}%".format((rev/TESTE["Teste"].size/prob_relevante_teste)*100))


Probabilidade de ser relevante segundo o algoritmo: 28.499999999999996%
Probabilidade de ser irrelevante segundo o algoritmo: 71.5%
Probabilidade de ser relevante pela nossa classificação: 44.50%
Probabilidade de ser irrelevante pela nossa classificação: 55.50%
Acurácia do algoritmo: 64.04%


___
### Concluindo

#### Validação da acurácia 

In [41]:
Xt = TREINAMENTO['Treinamento']
y = TREINAMENTO['Classificacao']

cnt = CountVectorizer()
X = cnt.fit_transform(Xt)
nb = MultinomialNB(alpha=1e-10)
nb.fit(X, y)

Xt_test = TESTE['Teste']
y_test = TESTE['Classificacao']

X_test = cnt.transform(Xt_test)
ypred = nb.predict(X_test)

acuracia_biblioteca = accuracy_score(y_test, ypred)

print("Acurácia do resultado segundo a biblioteca: {:.2f}%".format(acuracia_biblioteca))

Acurácia do resultado segundo a biblioteca: 0.65%


## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**